# Описание задачи

Ссылка на данные: https://ods.ai/competitions/nlp-receipts/data

Данные чеков ОФД содержат детальную информацию о тратах клиентов. Они помогают улучшать качество моделей кредитного скоринга и склонности к банковским продуктам, а также улучшать пользовательский опыт за счет структуризации трат клиентов в мобильном приложении. Однако работа с этим источником затрудняется его неструктурированностью: вся информация о купленном товаре лежит в одной строке произвольного формата.

В предположении что каждая чековая позиция описывает какой-либо товар, наименование этого товара, а также его бренд, являются главной информацией, которую можно извлечь из чека. По итогу задача структуризации этих данных ограничивается выделением и нормализацией брендов и товаров.

Участникам соревнования предоставляются два датасета с чековыми позициями, размеченный и неразмеченный:

В размеченном датасете для каждой чековой позиции указаны нормализованные бренды и товары входящие в нее в исходном виде.
В неразмеченном датасете даны только сами чековые позиции.

# Import

In [16]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
from catboost import Pool

import warnings
warnings.filterwarnings('ignore')

RAND = 10

In [2]:
train_supervised_path = 'data/train_supervised_dataset.csv'
train_unsupervised_path = 'data/train_unsupervised_dataset.csv'
test_path = 'data/test_dataset.csv'

In [3]:
df_sup = pd.read_csv(train_supervised_path)

In [4]:
df_unsup = pd.read_csv(train_unsupervised_path)

In [5]:
df_test = pd.read_csv(test_path)

In [6]:
df_sup.head()

,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


In [7]:
df_unsup.head()

,id,name
0,0,Зубная щетка Орал Би Три эффект Деликатное отб...
1,1,салфетки VISTER влажные для ко
2,2,Платье женское DR8517K 7Л8999 Светло-серый 449...
3,3,"ЛАКОМСТВО ""ДЕРЕВЕНСКИЕ ЛАКОМСТВА"" д/собак мини..."
4,4,Суппорт гитарный Ergo Play Troster


In [8]:
df_test.head()

,id,name
0,0,"469-210 ЕРМАК Клей универсальный, 15мл, блистер"
1,1,Торт СЛАДУШКА Зимняя вишня 700г
2,2,"Смеситель ""CALORIE"" 1023 А06 д/кухни"
3,3,Лимон 50гр БАР
4,4,"Коньяк САРАДЖИШВИЛИ 5 лет 0,5л Грузия"


In [9]:
df_sup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  int64 
 1   name    25000 non-null  object
 2   good    24181 non-null  object
 3   brand   16495 non-null  object
dtypes: int64(1), object(3)
memory usage: 781.4+ KB


In [10]:
df_sup.isna().sum()

id          0
name        0
good      819
brand    8505
dtype: int64

In [11]:
print(f"Доля пропусков в good: {round(df_sup['good'].isna().sum()/df_sup.shape[0]*100, 3)}%")
print(f"Доля пропусков в brand: {round(df_sup['brand'].isna().sum()/df_sup.shape[0]*100, 3)}%")

Доля пропусков в good: 3.276%
Доля пропусков в brand: 34.02%


In [12]:
# заполняет пропуски
for i in df_sup.columns:
    if i == 'good':
        df_sup['good'].fillna('None', inplace=True)
    elif i == 'brand':
        df_sup['brand'].fillna('None', inplace=True)
        
df_sup.isna().sum()   

id       0
name     0
good     0
brand    0
dtype: int64

In [13]:
df_sup.describe(include='object')

,name,good,brand
count,25000,25000,25000
unique,24994,2820,6976
top,Корм д/кошек Friskies кролик-ово,None,None
freq,2,819,8505


# Modeling

In [15]:
X1 = df_sup.drop(['good', 'id'], axis=1)
y1 = df_sup['good']
X2 = df_sup.drop(['brand', 'id'], axis=1)
y2 = df_sup['brand']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1,
                                                        y1,
                                                        test_size=0.2,
                                                        shuffle=True,
                                                        random_state=RAND)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2,
                                                        y2,
                                                        test_size=0.2,
                                                        shuffle=True,
                                                        random_state=RAND)

eval_set1 = [(X1_test, y1_test)]
eval_set2 = [(X2_test, y2_test)]

In [ ]:
%%time
clf1 = CatBoostClassifier(random_state=RAND, eval_metric="AUC")

clf1.fit(X1_train,
         y1_train,
         text_features=['name', 'brand'],
         verbose=False)